In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import warnings
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
from qiskit.circuit import Parameter
import torch 
import numpy as np
from pyscf import gto, scf
import scipy.sparse as sp
import matplotlib.pyplot as plt
from scipy.sparse.linalg import eigsh,eigs
import scipy.linalg as la
import scipy.sparse as sp


warnings.filterwarnings('ignore')

In [2]:
import numpy as np
from itertools import product

def generate_random_couplings(N):
    """ Generate random couplings J_ij for the SYK model. """
    couplings = np.random.randn(N, N, N, N)
    return couplings

def majorana_fermions(N):
    """ Construct Majorana fermion operators using tensor products of Pauli matrices. """
    sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
    sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)

    majorana_ops = []
    for i in range(N):
        op = 1
        for j in range(N):
            if j == i:
                op = np.kron(op, sigma_y)
            else:
                op = np.kron(op, sigma_z)
        majorana_ops.append(op)
    return majorana_ops

def construct_SYK_hamiltonian(N):
    """ Construct the SYK Hamiltonian matrix. """
    J = generate_random_couplings(N)
    majorana_ops = majorana_fermions(N)
    H = np.zeros((2**N, 2**N), dtype=complex)

    for i, j, k, l in product(range(N), repeat=4):
        if len({i, j, k, l}) == 4:  # Ensure all indices are different
            H += J[i, j, k, l] * majorana_ops[i] @ majorana_ops[j] @ majorana_ops[k] @ majorana_ops[l]
    return H

# Example with N=4
N = 4
H_SYK = torch.tensor(construct_SYK_hamiltonian(N))
print("Hamiltonian Matrix:\n", H_SYK)

Hamiltonian Matrix:
 tensor([[0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j,
         0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j,
         0.0000+0.j, 1.8359+0.j],
        [0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j,
         0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j,
         1.8359+0.j, 0.0000+0.j],
        [0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j,
         0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 1.8359+0.j,
         0.0000+0.j, 0.0000+0.j],
        [0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j,
         0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 1.8359+0.j, 0.0000+0.j,
         0.0000+0.j, 0.0000+0.j],
        [0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j,
      

In [9]:
import numpy as np
from itertools import product

def generate_random_couplings(N):
    """ Generate random couplings J_ij for the SYK model. """
    couplings = np.random.randn(N, N, N, N)
    return couplings

def majorana_fermions(N):
    """ Construct Majorana fermion operators using tensor products of Pauli matrices. """
    sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
    sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)

    majorana_ops = []
    for i in range(N):
        op = 1
        for j in range(N):
            if j == i:
                op = np.kron(op, sigma_y)
            else:
                op = np.kron(op, sigma_z)
        majorana_ops.append(op)
    return majorana_ops

def construct_SYK_hamiltonian(N, noise_strength=0.01):
    """ Construct the SYK Hamiltonian matrix with added random noise. """
    J = generate_random_couplings(N)
    majorana_ops = majorana_fermions(N)
    H = np.zeros((2**N, 2**N), dtype=complex)

    for i, j, k, l in product(range(N), repeat=4):
        if len({i, j, k, l}) == 4:  # Ensure all indices are different
            H += J[i, j, k, l] * majorana_ops[i] @ majorana_ops[j] @ majorana_ops[k] @ majorana_ops[l]

    # Add random noise to the Hamiltonian
    noise = np.random.normal(0, noise_strength, H.shape)
    H += noise

    return H

# Example with N=4
N = 4
H_SYK_noisy = torch.tensor(construct_SYK_hamiltonian(N))
print("Noisy Hamiltonian Matrix:\n", H_SYK_noisy)

Noisy Hamiltonian Matrix:
 tensor([[ 9.1379e-03+0.j, -1.9437e-02+0.j, -5.6738e-03+0.j,  7.3971e-03+0.j,
         -7.6421e-03+0.j, -7.5767e-03+0.j,  1.2343e-02+0.j, -2.6630e-04+0.j,
          1.0912e-03+0.j, -6.8230e-04+0.j,  2.6848e-02+0.j, -5.3419e-03+0.j,
         -8.6729e-03+0.j, -1.2634e-02+0.j,  5.8061e-03+0.j, -4.2408e+00+0.j],
        [ 7.3836e-03+0.j,  2.5376e-03+0.j, -3.6149e-03+0.j,  3.2615e-03+0.j,
          4.1154e-03+0.j, -6.9074e-03+0.j, -9.4188e-03+0.j, -1.4660e-03+0.j,
          6.1485e-03+0.j,  2.3172e-03+0.j,  5.5209e-03+0.j, -2.1115e-03+0.j,
          4.8999e-03+0.j,  5.0888e-03+0.j, -4.2305e+00+0.j,  3.2622e-03+0.j],
        [ 8.7660e-03+0.j, -5.5933e-04+0.j,  3.7847e-03+0.j, -8.3432e-03+0.j,
         -9.8304e-03+0.j,  1.5284e-02+0.j,  2.0003e-02+0.j, -7.5374e-03+0.j,
          3.5444e-03+0.j, -4.7364e-03+0.j,  1.4806e-02+0.j, -7.9565e-03+0.j,
          2.5742e-03+0.j, -4.2078e+00+0.j,  9.2381e-03+0.j, -4.8517e-03+0.j],
        [-8.1047e-03+0.j, -5.7099e-03+0.j, -6.

In [10]:
H_SYK_noisy

tensor([[ 9.1379e-03+0.j, -1.9437e-02+0.j, -5.6738e-03+0.j,  7.3971e-03+0.j,
         -7.6421e-03+0.j, -7.5767e-03+0.j,  1.2343e-02+0.j, -2.6630e-04+0.j,
          1.0912e-03+0.j, -6.8230e-04+0.j,  2.6848e-02+0.j, -5.3419e-03+0.j,
         -8.6729e-03+0.j, -1.2634e-02+0.j,  5.8061e-03+0.j, -4.2408e+00+0.j],
        [ 7.3836e-03+0.j,  2.5376e-03+0.j, -3.6149e-03+0.j,  3.2615e-03+0.j,
          4.1154e-03+0.j, -6.9074e-03+0.j, -9.4188e-03+0.j, -1.4660e-03+0.j,
          6.1485e-03+0.j,  2.3172e-03+0.j,  5.5209e-03+0.j, -2.1115e-03+0.j,
          4.8999e-03+0.j,  5.0888e-03+0.j, -4.2305e+00+0.j,  3.2622e-03+0.j],
        [ 8.7660e-03+0.j, -5.5933e-04+0.j,  3.7847e-03+0.j, -8.3432e-03+0.j,
         -9.8304e-03+0.j,  1.5284e-02+0.j,  2.0003e-02+0.j, -7.5374e-03+0.j,
          3.5444e-03+0.j, -4.7364e-03+0.j,  1.4806e-02+0.j, -7.9565e-03+0.j,
          2.5742e-03+0.j, -4.2078e+00+0.j,  9.2381e-03+0.j, -4.8517e-03+0.j],
        [-8.1047e-03+0.j, -5.7099e-03+0.j, -6.0593e-03+0.j,  1.2429e-02+0

In [14]:
# Define the classical encoder neural network
class ClassicalEncoder(nn.Module):
    def __init__(self):
        super(ClassicalEncoder, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(16, 8),  # First layer with 7 inputs and 14 outputs
            nn.ReLU(),         # Activation function
            #nn.Linear(28, 56), # Second layer with 14 inputs and 28 outputs
            #nn.ReLU(),         # Activation function
            #nn.Linear(56, 28), # Third layer with 28 inputs and 56 outputs
            #nn.ReLU(),         # Activation function
            #nn.Linear(28, 14), # Fourth layer reducing from 56 to 28 outputs
            #nn.ReLU(),         # Activation function
            nn.Linear(8, 4) # Fifth layer reducing from 28 to 14 outputs
        )
    
    def forward(self, x):
        return self.fc(x)

encoder = ClassicalEncoder()
#print("The encoder is: ", encoder)

# Define a function to execute the quantum circuit
def run_quantum_circuit(params):
    # Create a list of parameters for the quantum circuit
    theta = [Parameter(f'θ{i}') for i in range(4)]

    # add control rotation gates
    
    # Create a quantum circuit with 4 qubits and 4 classical bits
    qc = QuantumCircuit(4)
    
    # Apply initial Ry and Rx rotations
    for i in range(4):
        qc.rz(theta[i], i)
        qc.rx(theta[i], i)
        qc.rz(theta[i], i)

    qc.barrier()


    for i in range(4):
        qc.rxx(theta[i],0,1)
        qc.rxx(theta[i],1,2)
        qc.rxx(theta[i],2,3)
        qc.rxx(theta[i],0,3)
    

    qc.barrier()
    
    

    # Add measurements to all qubits
    qc.measure_all()
    
    # Bind the parameters to the values from the PyTorch model
    param_dict = {theta[i]: params[i].item() for i in range(4)}
    qc_bound = qc.bind_parameters(param_dict)
    
    # Print the quantum circuit
    #print(qc_bound)

    # If you want a visual diagram of the circuit, you can use:
    # circuit_drawer(qc_bound, output='mpl').show()

    # Execute the quantum circuit
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc_bound, backend, shots=1024)
    result = job.result()
    counts = result.get_counts(qc_bound)
    
    # Plot the histogram of results
    plot_histogram(counts)
    
    # Get the most common bitstring
    output_bitstring = max(counts, key=counts.get)
    
    # Convert bitstring to numpy array of integers
    output_data = np.array([int(bit) for bit in output_bitstring])
    
    # Convert to PyTorch tensor
    output_tensor = torch.tensor(output_data, dtype=torch.float32)
    
    return output_tensor

# Define the classical decoder neural network
class ClassicalDecoder(nn.Module):
    def __init__(self):
        super(ClassicalDecoder, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 8),    # First layer with 4 inputs and 8 outputs
            #nn.ReLU(),          # Activation function
            #nn.Linear(8, 16),   # Second layer with 8 inputs and 16 outputs
            #nn.ReLU(),          # Activation function
            #nn.Linear(16, 32),  # Third layer with 16 inputs and 32 outputs
            #nn.ReLU(),          # Activation function
            #nn.Linear(32, 64),
            #nn.ReLU(),
            #nn.Linear(64, 32),  # Fourth layer reducing from 32 to 16 outputs
            nn.ReLU(),          # Activation function
            nn.Linear(8, 16)
        )
    
    def forward(self, x):
        return self.fc(x)

decoder = ClassicalDecoder()
#print("The decoder is: ", decoder)


class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        self.encoder = ClassicalEncoder()
        self.decoder = ClassicalDecoder()
        self.qcircuit = run_quantum_circuit

    def forward(self, x):
        encoded = self.encoder(x)
        quantum_result = self.qcircuit(encoded)
        decoded = self.decoder(quantum_result)
        return decoded

# Initialize the model
model = HybridModel()



In [15]:
def energy_expectation(output, hamiltonian):
    # ... [previous code] ...

    # Convert hamiltonian to double
    hamiltonian = hamiltonian.type(torch.double)

    # Convert output to double
    wavefunction = output.type(torch.double)

    # Normalize the wavefunction
    norm_wavefunction = wavefunction / torch.sqrt(torch.sum(torch.abs(wavefunction)**2))

    # Calculate the energy expectation value
    energy = torch.vdot(norm_wavefunction, torch.mv(hamiltonian, norm_wavefunction)).real

    return energy


In [17]:
# Sample input
input_data = torch.rand(16, requires_grad=True)  # Example input
#input_data= torch.tensor([ 0.3679, -0.0602,  0.6200,  0.1083, -0.0054,  0.0107,  0.1241, 0.3679, -0.0602,  0.6200,  0.1083, -0.0054,  0.0107,  0.1241])

# Optimization setup
#print("The model parameters are: ", model.parameters)
optimizer = optim.Adam(model.parameters(), lr=0.01)
num_epochs = 100
loss_values = []

for epoch in range(num_epochs):
    optimizer.zero_grad()            # Clear existing gradients
    output = model(input_data)       # Forward pass

    # Ensure output requires grad
    if not output.requires_grad:
        raise RuntimeError("Output does not require gradients. Check model implementation.")

    # Calculate the loss
    #initial_hamiltonian = hamiltonian_initial_module.mf.get_hcore()
    #final_hamiltonian = hamiltonian_final_module.mf.get_hcore()
    loss = energy_expectation(output,H_SYK_noisy)
    # Check if loss requires grad
    if not loss.requires_grad:
        raise RuntimeError("Loss does not require gradients. Check energy_expectation implementation.")

    loss.backward()                  # Backward pass
    optimizer.step()                 # Update parameters
    loss_values.append(loss.item())  # Store loss for plotting
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")


Epoch 1/100, Loss: -4.237978873244355
Epoch 2/100, Loss: -4.235712369321642
Epoch 3/100, Loss: -4.235945975858907
Epoch 4/100, Loss: -4.238646529429964
Epoch 5/100, Loss: -4.237880484138767
Epoch 6/100, Loss: -4.237878946102789
Epoch 7/100, Loss: -4.24059009044586
Epoch 8/100, Loss: -4.241738898146679
Epoch 9/100, Loss: -4.241014457360539
Epoch 10/100, Loss: -4.241093466268914
Epoch 11/100, Loss: -4.242072524051978
Epoch 12/100, Loss: -4.242835706490813
Epoch 13/100, Loss: -4.243289832684481
Epoch 14/100, Loss: -4.243657568049063
Epoch 15/100, Loss: -4.24380109497855
Epoch 16/100, Loss: -4.243899401189233
Epoch 17/100, Loss: -4.2443039110757095
Epoch 18/100, Loss: -4.244909554303396
Epoch 19/100, Loss: -4.245288378650098
Epoch 20/100, Loss: -4.2453554079895515
Epoch 21/100, Loss: -4.2454001545592615
Epoch 22/100, Loss: -4.245652867083159
Epoch 23/100, Loss: -4.24601505133341
Epoch 24/100, Loss: -4.246290462907226
Epoch 25/100, Loss: -4.2464310822214895
Epoch 26/100, Loss: -4.2465483878

In [13]:
import scipy.linalg as la

def find_lowest_eigenvalue(matrix):
    # Compute all eigenvalues, but only the first eigenvectors
    eigenvalues, _ = la.eigh(matrix, eigvals=(0, 0))
    return eigenvalues[0]

# Assuming large_matrix is your matrix
lowest_eigenvalue = find_lowest_eigenvalue(H_SYK_noisy)
print("Lowest Eigenvalue:", lowest_eigenvalue)


Lowest Eigenvalue: -4.265873872480003


In [ ]:
# for the no pqc ground state approximation was: -4.239352522251876
# for the pqc ground state: -4.253164531632698
